# Demonstration of the implementation concept of CDT

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pylab as plt
import collections
collections.Iterable = collections.abc.Iterable

In [ ]:
# from bmcs_shear.api import DICGridTri
# dic_grid = DICGridTri(U_factor=100, dir_name='B10_TV1', t=1, padding=40, d_x=3, d_y=3, n_T=40)
# dic_grid.read_beam_design()
# np.savez_compressed('X_TPa', X_TPa=dic_grid.X_TQa[:,:,:-1])

## Input format

In [ ]:
loaded = np.load('X_TPa.npz')
X_TPa = loaded['X_TPa']

In [ ]:
X_0Pa = X_TPa[0]
U_TPa = X_TPa - X_0Pa[None,...]
n_T = len(U_TPa)

## Grid definition

In [ ]:
pad_a = np.array([40, 40])
X_min_a = np.array([np.min(X_0Pa[:,a]+pad_a[a]) for a in (0,1)]) # left&bottom
X_max_a = np.array([np.max(X_0Pa[:,a]-pad_a[a]) for a in (0,1)]) # right&top

In [ ]:
L_a = X_max_a - X_min_a # frame dimensions
delta_a = 4 # 4 mm distances
n_I, n_J = np.array( L_a / delta_a, dtype=np.int_ )
delta_X_a = [L_a[0]/n_I, L_a[1]/n_J]
n_a = 2
n_I, n_J

Given a column index $I = [0, n_I-1\}$, row index $J = [0, n_J-1]$ and direction index $a = [0,1]$ 
the index expression
$$
\mathcal{G}_{IJa} = (1-a)I + a J
$$
introduces a grid index map rendering the horizontal/column indexes of individual grid nodes for $a=0$ and vertical / row indexes for $a=1$.

$$
 a = 0 \rightarrow (\zeta = 1, \eta = 0)
$$
$$
 a = 1 \rightarrow (\zeta = 0, \eta = 1)
$$
$$
\zeta = 1-a
$$
$$
\eta = a
$$

In [ ]:
I,J,a = [np.arange(n) for n in (n_I,n_J,n_a)]
G_aIJ = (np.einsum('a, I->aI', (1-a), I)[:,:, None] + 
         np.einsum('a, J->aJ', a, J)[:,None, :])

Given the step length 
$$
\Delta X_a = \left[\frac{L_x}{n_I}, \frac{L_y}{n_J} \right],
$$
the distance between grid points is
The coordinates of all nodes are expressed as
$$
X_{aIJ} = \Delta X_{a} \, \mathcal{G}_{aIJ}
$$

In [ ]:
X_aIJ = X_min_a[:,None,None] + np.einsum('aIJ,a->aIJ', G_aIJ, delta_X_a)
X_IJa = np.einsum('aIJ->IJa', X_aIJ)

In [ ]:
from scipy.spatial import Delaunay
from scipy.interpolate import \
    LinearNDInterpolator as LNDI
tri = Delaunay(X_0Pa)

Use an interpolator over the Delaunay triangulation to obtain the values over a regular grid $X_{aIJ}$

In [ ]:
U_TIJa = np.array([
    LNDI(tri, U_TPa[T])(*X_aIJ) for T in range(n_T)
])

The enumeration of nodes within a single element is defined for the $\xi$ and $\eta$ directions consistently with the enumeration of the $\mathcal{G}_{aIJ}$ grid by setting 
$$
{g}_{aij} = \mathcal{G}_{a; I=i; J=i}, \; i \in (0,1) \times (0,1)
$$

In [ ]:
g_aij = G_aIJ[:,:2,:2]

By introducing the element indexes $E = [0, n_I-2]$ and $F = [0, n_J-2]$ in the 
horizontal and vertical direction, we can introduce the index map identifying
the local element nodes enumerated counter clock-wise in each element of the grid as
$$
\mathcal{H}_{aEiFj} = \mathcal{G}_{aEF} + g_{aij}
$$

In [ ]:
G_aEF = G_aIJ[:,:-1,:-1]

In [ ]:
H_aEiFj = G_aEF[:,:,None,:,None] + g_aij[:,None,:,None,:]

$$
  X_{EiFja} = X_{I=\mathcal{H}_{0EiFj}, I=\mathcal{H}_{1EiFj}}
$$

In [ ]:
X_EiFja = X_IJa[(*H_aEiFj,)]
U_TEiFja = U_TIJa[(slice(None), *H_aEiFj)]

## Nodal coordinates and quadrature points of an element

In [ ]:
delta_rs = np.eye(2, dtype=np.int_)

$$
\xi_{rij} = 2 X_{a;E=0;i;F=0;j} - 1
$$

In [ ]:
xi_rij = (H_aEiFj[:,0,:,0,:] * 2) - 1

$$
 \eta_{rmn} = \frac{1}{\sqrt{3}} \xi_{r;i=m;j=n}, \; m, n \in (0,1) \times (0,1)
$$

In [ ]:
eta_rmn = 3**(-1/2) * xi_rij

## Bilinear Lagrange shape functions

### Dimensional directions explicitly referenced in the product expression
$$
N_{ijmn} = \frac{1}{4}(1 + \eta_{r=0;mn} \xi_{r=0;ij})\,(1 + \eta_{r=1;mn} \xi_{r=1;ij})
$$

In [ ]:
N1_ijmn = (
    (1 + np.einsum('mn,ij->mnij', eta_rmn[0], xi_rij[0]))* 
    (1 + np.einsum('mn,ij->mnij', eta_rmn[1], xi_rij[1]))
) / 4

### Dimensional directions included in the index operator
$$
N_{ij}(\eta_r) 
=
\frac{1}{4}(
1 + \eta_0 \xi_{0ij} + \eta_1 \xi_{1ij} + \eta_0 \xi_{0ij} \eta_1 \xi_{1ij}
)
$$

$$
N_{ijmn}
=
\frac{1}{4}\left(
1 + \eta_{rmn} \xi_{rij} + \frac{1}{2}(1 - \delta_{rs}) \eta_{smn} \xi_{sij} \eta_{rmn} \xi_{rij}
\right)
$$

In [ ]:
N_ijmn = (1 + 
  np.einsum('rmn,rij->ijmn', eta_rmn, xi_rij) +
  np.einsum('rs,smn,sij,rmn,rij->ijmn', (1-delta_rs), eta_rmn, xi_rij, eta_rmn, xi_rij) / 2
)/4
np.sum(N_ijmn - N1_ijmn)

### Derivatives of the shape functions w.r.t. parametric coordinates

$$
\frac{\partial N_{ij} }{\partial \eta_0}
= 
\frac{1}{4}( \xi_{0ij} + \eta_1 \xi_{1ij} ), \;\;
\frac{\partial N_{ij} }{\partial \eta_1}
= 
\frac{1}{4}( \xi_{1ij} + \eta_0 \xi_{0ij} )
$$

$$
N_{ij,s}(\eta_r)
= 
\frac{1}{4}( \xi_{sij} +
(1-\delta_{rs}) \xi_{sij} \eta_{r} \xi_{rij}
)
$$

$$
N_{ijmn,s}
= 
\frac{1}{4}
\left[ \xi_{sij} +
(1-\delta_{rs}) \, \xi_{sij} \eta_{rmn} \xi_{rij}
\right]
$$

In [ ]:
dN_sijmn = (
    xi_rij[:,:,:,None,None] + 
    np.einsum('rs,sij,rmn,rij->sijmn', (1 - delta_rs), xi_rij, eta_rmn, xi_rij)
) / 4

## Kinematic operator

$$
J_{EmFnas} = N_{ijmn,s} X_{EiFja}
$$

In [ ]:
J_EmFnas = np.einsum(
 'sijmn,EiFja->EmFnas',
 dN_sijmn, X_EiFja
)

In [ ]:
inv_J_EmFnsa = np.linalg.inv(J_EmFnas)

In [ ]:
delta_ab = np.eye(2)
Diff1_abcd = 0.5 * (
    np.einsum('ac,bd->abcd', delta_ab, delta_ab) +
    np.einsum('ad,bc->abcd', delta_ab, delta_ab)
)

$$
B_{EiFjmnabc} = D_{abcd} N_{ijmn,s} J^{-1}_{EmFnsd}
$$

In [ ]:
B_EiFjmnabc = np.einsum(
    'abcd,sijmn,EmFnsd->EiFjmnabc',
    Diff1_abcd, dN_sijmn, inv_J_EmFnsa
)

$$
\varepsilon_{EmFnab} = B_{EiFjmnabc} U_{EiFjc}
$$

In [ ]:
eps_TEmFnab = np.einsum(
    'EiFjmnabc,TEiFjc->TEmFnab',
    B_EiFjmnabc, U_TEiFja
)

## Scalar damage model

### Equivalent strain

In [ ]:
eps_TEmFna, _ = np.linalg.eig(eps_TEmFnab)
kappa_TEmFn = np.max(eps_TEmFna, axis=-1)

### Scalar damage model

In [ ]:
eps_0=1e-3
eps_f=0.01
I = np.where(kappa_TEmFn>=eps_0)
omega_TEmFn = np.zeros_like(kappa_TEmFn)
omega_TEmFn[I] = 1.0-(eps_0/kappa_TEmFn[I]*np.exp(
    -(kappa_TEmFn[I]-eps_0)/(eps_f-eps_0)))

### Constitutive law

In [ ]:
E_ = 28000
nu_ = 0.18
la = E_*nu_/((1+ nu_)*(1-2*nu_))
mu = E_/(2+2*nu_)
delta = np.eye(2)
D_abef = (
 np.einsum(',ij,kl->ijkl',la,delta,delta)+
 np.einsum(',ik,jl->ijkl',mu,delta,delta)+
 np.einsum(',il,jk->ijkl',mu,delta,delta))
sig_TEmFnab = np.einsum('...,abef,...ef -> ...ab', 
 omega_TEmFn, D_abef, eps_TEmFnab)

## Plot localized damage field

In [ ]:
X_aEmFn = np.einsum('ijmn,EiFja->aEmFn', N_ijmn, X_EiFja)
X_aKL = X_aEmFn.reshape(-1,(n_I-1)*2, (n_J-1)*2)

In [ ]:
omega_TKL = omega_TEmFn.reshape(-1,(n_I-1)*2, (n_J-1)*2)

In [ ]:
fig = plt.figure(figsize=(8,5))
ax1, ax2 = fig.subplots(2,1)
for ax, T in zip([ax1, ax2], [10, 30]):
    ax.contourf(X_aKL[0], X_aKL[1], omega_TKL[T], cmap='BuPu',
                levels=[0.0, 0.2, 0.4, 0.8, 0.9, 1.0], vmin=0, vmax=1)
    ax.axis('equal');
    ax.axis('off');